<a href="https://colab.research.google.com/github/grjadhav409/Enthalpy-prediction-ML/blob/main/2_1_SMILES_cleaning(rdkit).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!time bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c rdkit rdkit

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
import numpy as np
import pandas as pd
import tqdm.auto as tqdm

from matplotlib import pyplot as plt
import seaborn as sns

import rdkit, rdkit.Chem.AllChem, rdkit.Chem.Crippen
import rdkit.Chem as Chem
from rdkit.Chem import Draw

import torch
import sklearn.linear_model, sklearn.model_selection

In [ ]:
df= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/physical property predictions/Enthalpy/enthalpy data/df_smiles.csv")
df

In [ ]:
df2 = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df2

In [ ]:
smiles_old= df.SMILES.tolist()
smiles_old

In [ ]:
smiles_list_new= []

for i in smiles_old:
  x = rdkit.Chem.MolFromSmiles(i)
  smiles_list_new.append(x)

df3= pd.DataFrame(data= smiles_list_new)
df3

In [ ]:
df4 = pd.concat([df2, df3], axis=1)
df4

,name,hfg,SMILES,0
0,perfluoropropane,-1784.70,FC(F)(F)C(F)(F)C(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090f0d50>
1,Silicon tetrafluoride,-1615.00,[F-].[F-].[F-].[F-].[Si+4],<rdkit.Chem.rdchem.Mol object at 0x7f8a090bf990>
2,Phosphorus pentafluoride,-1593.30,F[P](F)(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090a3b20>
3,Diboron tetrafluoride,-1438.00,FB(F)B(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090acad0>
4,hexafluoroethane,-1343.90,FC(F)(F)C(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090aca30>
...,...,...,...,...
852,Boron silicide,698.00,B[SiH3],<rdkit.Chem.rdchem.Mol object at 0x7f8a09057ee0>
853,silicon monocarbide,719.65,[C-]#[Si+],<rdkit.Chem.rdchem.Mol object at 0x7f8a09057f30>
854,carbon trimer,820.06,[CH2]=[C]=[CH2],<rdkit.Chem.rdchem.Mol object at 0x7f8a09057f80>
855,Hydrogen cation,1494.67,[H],<rdkit.Chem.rdchem.Mol object at 0x7f8a09058030>


In [ ]:
df4.columns

Index(['name', 'hfg', 'SMILES', 0], dtype='object')

In [ ]:
df5= df4.rename(columns={"name": "name", "hfg": "hfg", "SMILES": "smiles", 0: "rdkit_object"})
df5

,name,hfg,smiles,rdkit_object
0,perfluoropropane,-1784.70,FC(F)(F)C(F)(F)C(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090f0d50>
1,Silicon tetrafluoride,-1615.00,[F-].[F-].[F-].[F-].[Si+4],<rdkit.Chem.rdchem.Mol object at 0x7f8a090bf990>
2,Phosphorus pentafluoride,-1593.30,F[P](F)(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090a3b20>
3,Diboron tetrafluoride,-1438.00,FB(F)B(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090acad0>
4,hexafluoroethane,-1343.90,FC(F)(F)C(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090aca30>
...,...,...,...,...
852,Boron silicide,698.00,B[SiH3],<rdkit.Chem.rdchem.Mol object at 0x7f8a09057ee0>
853,silicon monocarbide,719.65,[C-]#[Si+],<rdkit.Chem.rdchem.Mol object at 0x7f8a09057f30>
854,carbon trimer,820.06,[CH2]=[C]=[CH2],<rdkit.Chem.rdchem.Mol object at 0x7f8a09057f80>
855,Hydrogen cation,1494.67,[H],<rdkit.Chem.rdchem.Mol object at 0x7f8a09058030>


In [ ]:
df5.head(30)

,name,hfg,smiles,rdkit_object
0,perfluoropropane,-1784.70,FC(F)(F)C(F)(F)C(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090f0d50>
1,Silicon tetrafluoride,-1615.00,[F-].[F-].[F-].[F-].[Si+4],<rdkit.Chem.rdchem.Mol object at 0x7f8a090bf990>
2,Phosphorus pentafluoride,-1593.30,F[P](F)(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090a3b20>
3,Diboron tetrafluoride,-1438.00,FB(F)B(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090acad0>
4,hexafluoroethane,-1343.90,FC(F)(F)C(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090aca30>
5,Phosphoryl fluoride,-1252.00,F[P](F)(F)=O,<rdkit.Chem.rdchem.Mol object at 0x7f8a090c24e0>
6,Sulfur Hexafluoride,-1220.47,F[S](F)(F)(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a09085a80>
7,Aluminum trifluoride,-1209.32,[F-].[F-].[F-].[Al+3],<rdkit.Chem.rdchem.Mol object at 0x7f8a09085f30>
8,trifluorosilane,-1200.81,F[SiH](F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8aca89a3f0>
9,Germanium tetrafluoride,-1190.20,[F-].[F-].[F-].[F-].[Ge+4],<rdkit.Chem.rdchem.Mol object at 0x7f8a090980d0>


In [ ]:
df6= df5.dropna()
df5.shape,df6.shape

((857, 4), (854, 4))

In [ ]:
df6

,name,hfg,smiles,rdkit_object
0,perfluoropropane,-1784.70,FC(F)(F)C(F)(F)C(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090f0d50>
1,Silicon tetrafluoride,-1615.00,[F-].[F-].[F-].[F-].[Si+4],<rdkit.Chem.rdchem.Mol object at 0x7f8a090bf990>
2,Phosphorus pentafluoride,-1593.30,F[P](F)(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090a3b20>
3,Diboron tetrafluoride,-1438.00,FB(F)B(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090acad0>
4,hexafluoroethane,-1343.90,FC(F)(F)C(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090aca30>
...,...,...,...,...
852,Boron silicide,698.00,B[SiH3],<rdkit.Chem.rdchem.Mol object at 0x7f8a09057ee0>
853,silicon monocarbide,719.65,[C-]#[Si+],<rdkit.Chem.rdchem.Mol object at 0x7f8a09057f30>
854,carbon trimer,820.06,[CH2]=[C]=[CH2],<rdkit.Chem.rdchem.Mol object at 0x7f8a09057f80>
855,Hydrogen cation,1494.67,[H],<rdkit.Chem.rdchem.Mol object at 0x7f8a09058030>


In [ ]:
#reset index
df6.reset_index(drop=True, inplace=True)

In [ ]:
smiles_cleaned= []
smiles_raw= []
for i in df6.smiles:
  x = Chem.MolToSmiles(Chem.MolFromSmiles(i))
  smiles_cleaned.append(x)
for i in range(853):
  y = df6.smiles[i]
  smiles_raw.append(y)

In [ ]:
smiles_cleaned = pd.DataFrame(data= smiles_cleaned)
smiles_cleaned.drop(smiles_cleaned.tail(1).index,inplace=True) # drop last n rows
smiles_raw = pd.DataFrame(data= smiles_raw)

In [ ]:
df8= pd.concat([smiles_raw,smiles_cleaned], axis=1)

In [ ]:
df8.columns = ['smiles', 'smiles_cleaned']

In [ ]:
df8

,smiles,smiles_cleaned
0,FC(F)(F)C(F)(F)C(F)(F)F,FC(F)(F)C(F)(F)C(F)(F)F
1,[F-].[F-].[F-].[F-].[Si+4],[F-].[F-].[F-].[F-].[Si+4]
2,F[P](F)(F)(F)F,FP(F)(F)(F)F
3,FB(F)B(F)F,FB(F)B(F)F
4,FC(F)(F)C(F)(F)F,FC(F)(F)C(F)(F)F
...,...,...
848,[Al]#C[Al]=C=[Al]C#[Al],[Al]#C[Al]=C=[Al]C#[Al]
849,B[SiH3],B[SiH3]
850,[C-]#[Si+],[C-]#[Si+]
851,[CH2]=[C]=[CH2],C=C=C


In [ ]:
df9 = pd.merge(df6, df8, on="smiles")
#reset index
df9.reset_index(drop=True, inplace=True)
df9.drop_duplicates()
df9

,name,hfg,smiles,rdkit_object,smiles_cleaned
0,perfluoropropane,-1784.70,FC(F)(F)C(F)(F)C(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090f0d50>,FC(F)(F)C(F)(F)C(F)(F)F
1,Silicon tetrafluoride,-1615.00,[F-].[F-].[F-].[F-].[Si+4],<rdkit.Chem.rdchem.Mol object at 0x7f8a090bf990>,[F-].[F-].[F-].[F-].[Si+4]
2,Phosphorus pentafluoride,-1593.30,F[P](F)(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090a3b20>,FP(F)(F)(F)F
3,Diboron tetrafluoride,-1438.00,FB(F)B(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090acad0>,FB(F)B(F)F
4,hexafluoroethane,-1343.90,FC(F)(F)C(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090aca30>,FC(F)(F)C(F)(F)F
...,...,...,...,...,...
924,Tetracyanoethylene,688.10,N#CC(C#N)=C(C#N)C#N,<rdkit.Chem.rdchem.Mol object at 0x7f8a09057e40>,N#CC(C#N)=C(C#N)C#N
925,Aluminum carbide,689.52,[Al]#C[Al]=C=[Al]C#[Al],<rdkit.Chem.rdchem.Mol object at 0x7f8a09057e90>,[Al]#C[Al]=C=[Al]C#[Al]
926,Boron silicide,698.00,B[SiH3],<rdkit.Chem.rdchem.Mol object at 0x7f8a09057ee0>,B[SiH3]
927,silicon monocarbide,719.65,[C-]#[Si+],<rdkit.Chem.rdchem.Mol object at 0x7f8a09057f30>,[C-]#[Si+]


In [ ]:
df10=df9.drop_duplicates()
df10.reset_index(drop=True, inplace=True)
df10.head(50)

,name,hfg,smiles,rdkit_object,smiles_cleaned
0,perfluoropropane,-1784.70,FC(F)(F)C(F)(F)C(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090f0d50>,FC(F)(F)C(F)(F)C(F)(F)F
1,Silicon tetrafluoride,-1615.00,[F-].[F-].[F-].[F-].[Si+4],<rdkit.Chem.rdchem.Mol object at 0x7f8a090bf990>,[F-].[F-].[F-].[F-].[Si+4]
2,Phosphorus pentafluoride,-1593.30,F[P](F)(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090a3b20>,FP(F)(F)(F)F
3,Diboron tetrafluoride,-1438.00,FB(F)B(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090acad0>,FB(F)B(F)F
4,hexafluoroethane,-1343.90,FC(F)(F)C(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a090aca30>,FC(F)(F)C(F)(F)F
5,Phosphoryl fluoride,-1252.00,F[P](F)(F)=O,<rdkit.Chem.rdchem.Mol object at 0x7f8a090c24e0>,O=P(F)(F)F
6,Sulfur Hexafluoride,-1220.47,F[S](F)(F)(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8a09085a80>,FS(F)(F)(F)(F)F
7,Aluminum trifluoride,-1209.32,[F-].[F-].[F-].[Al+3],<rdkit.Chem.rdchem.Mol object at 0x7f8a09085f30>,[Al+3].[F-].[F-].[F-]
8,trifluorosilane,-1200.81,F[SiH](F)F,<rdkit.Chem.rdchem.Mol object at 0x7f8aca89a3f0>,F[SiH](F)F
9,Germanium tetrafluoride,-1190.20,[F-].[F-].[F-].[F-].[Ge+4],<rdkit.Chem.rdchem.Mol object at 0x7f8a090980d0>,[F-].[F-].[F-].[F-].[Ge+4]


In [ ]:
Chem.MolToSmiles(Chem.MolFromSmiles("FBr(F)(F)(F)F"))

'FBr(F)(F)(F)F'

In [ ]:
df10.to_csv('smiles_cleaned.csv')
!cp smiles_cleaned.csv "/content/drive/MyDrive/Colab Notebooks/physical property predictions/Enthalpy/enthalpy data"